In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "cpu-cluster"

# Verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4, 
                                                           idle_seconds_before_scaledown=2400)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.core import ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
    "--C": uniform(0.1, 1.0),
    "--max_iter": choice(range(1, 100))
    }
)

# Specify a Policy
# The BanditPolicy basically states to check the job every 2 iterations.
# If the primary metric (defined later) falls outside of the top 10% range, Azure ML terminate the job.
# More accurately, any run whose best metric is less than (1/(1+0.1) or 91% of the best performing run will be terminated.
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=2)

# Create a SKLearn estimator for use with train.py
# https://docs.microsoft.com/ja-jp/python/api/azureml-train-core/azureml.train.sklearn.sklearn?view=azure-ml-py
est = SKLearn(source_directory="./",
            entry_script="train.py",
            compute_tarxsget="cpu_cluster")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling = ps,
                                    primary_metric_name = 'accuracy',
                                    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs = 100,
                                    max_concurrent_runs= 4,
                                    max_duration_minutes=10080,
                                    policy=policy,
                                    estimator=est,
                                    run_config=None,
                                    resume_from=None,
                                    resume_child_runs=None,
                                    pipeline=None,
                                    debug_flag=None)

In [ ]:
from azureml.widgets import RunDetails
# Submit your hyperdrive run to the experiment and show run details with the widget.
# https://github.com/MicrosoftDocs/azure-docs/blob/master/articles/machine-learning/how-to-tune-hyperparameters.md#submit-hyperparameter-tuning-experiment
hyperdrive_run = exp.submit(hyperdrive_config, show_output = False)
# cf. https://docs.microsoft.com/ja-jp/python/api/azureml-widgets/azureml.widgets.rundetails?view=azure-ml-py
# cf. https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/classification-credit-card-fraud/auto-ml-classification-credit-card-fraud.ipynb
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)

In [ ]:
import joblib
# Get your best run and save the model from that run.
# cf. https://docs.microsoft.com/ja-jp/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriverun?view=azure-ml-py#get-best-run-by-primary-metric-include-failed-true--include-canceled-true--include-resume-from-runs-true-----typing-union-azureml-core-run-run--nonetype-
best_run = hyperdrive_run.get_best_run_by_primary_metric(include_failed = False,
                                            include_canceled=False,
                                            include_resume_from_runs=True)
print(best_run.get_metris())
joblib.dump(best_run, './hyperdrive_best_run.joblib', compress = 3)

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

dataset_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = TabularDatasetFactory.from_delimited_files(dataset_url,
                                                    separator = ',',
                                                    encoding = 'utf8')

In [ ]:

from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)
training_data, validation_data = dataset.random_split(percentage=0.8, seed=1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
# cf. https://docs.microsoft.com/ja-jp/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    task = 'classification',
    primary_metric = 'accuracy',
    training_data = training_data,
    valudation_data = validation_data,
    label_column_name = 'y',
    n_cross_validations = 5)

In [2]:
# Submit your automl run

autoMLExp = Experiment(workspace=ws, name="udacity-project-automl")
automl_run = autoMLExp.submit(automl_config, show_output = False)

In [ ]:
# Retrieve and save your best automl model.
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output = True)

In [ ]:
automl_run = hyperdrive_run.get_best_run_by_primary_metric(include_failed = False,
                                            include_canceled=False,
                                            include_resume_from_runs=True)
print(automl_run.get_metris())
joblib.dump(automl_run, './automl_best_run.joblib', compress = 3)